In [1]:
import os, glob, cv2, math, csv, tqdm, random, keras, gc

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import tensorflow
import numpy as np
from PIL import Image
import util 
import tensorflow as tf
from keras.utils.training_utils import multi_gpu_model
from keras import backend as k
from keras import applications, optimizers, losses, layers
from keras.utils.training_utils import multi_gpu_model
from keras import layers ,losses
from keras.layers import Dropout, Input, Flatten, Dense, GlobalAveragePooling2D, Activation, LSTM, SeparableConv2D, TimeDistributed, GRU
from keras.models import Sequential, Model, load_model
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.layers.convolutional import Conv2D
%matplotlib inline
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [2]:
def data_list_load(seq, root):
    img_path_total = []
    lab_path_total = []
    for seq_ch in seq:
        img_dir = os.path.join(root, 'sequences', seq_ch,'image_2')
        seq_pose = seq_ch + '.txt'
        lab_dir = os.path.join(root, 'poses', seq_pose)
        lab_list = util.load_dof6_labels(lab_dir)
        lab_dir = os.path.join(root, 'poses', seq_pose)
        paths = sorted(glob.glob(img_dir+'/*.png'))

        for i in tqdm.tqdm(paths):
            img_time = [[paths[i], paths[i+1], paths[i+2], 
                         paths[i+3], paths[i+4], paths[i+5], 
                         paths[i+6], paths[i+7], paths[i+8], 
                         paths[i+9], paths[i+10]] 
                        for i in range(len(paths)-10)]
        img_path_total = img_path_total + img_time
        lab_path_total = lab_path_total + lab_list[9:]
    print(len(img_path_total), len(lab_path_total))
    return img_path_total, lab_path_total

def test_generator(total_img, total_lab, batch):
    total = []
    idx_batch = 0        
    while 1:
        bat_img = []
        bat_lab = []
        tmp_img = []       
        for j in range(10):
            img = cv2.resize(np.concatenate((Image.open(total_img[idx_batch][j]),Image.open(total_img[idx_batch][j+1])),axis = 2),(512,384))/255.
            tmp_img.append(img)
        lab = total_lab[idx_batch]
        bat_img.append(np.array(tmp_img))
        bat_lab.append(lab) 
        idx_batch = idx_batch + 1
        yield np.array(bat_img), [np.array(bat_lab)[:,0],
                                  np.array(bat_lab)[:,1],
                                  np.array(bat_lab)[:,2],
                                  np.array(bat_lab)[:,3],
                                  np.array(bat_lab)[:,4],
                                  np.array(bat_lab)[:,5]]

In [3]:
root = 'save_multi2'
model_dir = sorted(glob.glob(root+'/*.h5'))
model_dir[-1]

'save_multi2/gap_multi_lstm00000010.h5'

In [4]:
model = load_model(model_dir[-1])
batch = 1
test_seq = ['00']
test_root= '/data1/Kitti/odometry/'
test_list, test_lab = data_list_load(test_seq, test_root)
test = test_generator(test_list, test_lab, batch)

100%|██████████| 4541/4541 [00:27<00:00, 165.84it/s]

4531 4531


In [5]:
import time

In [6]:
predic = []
labels = []

for i in range(1):
    start = time.time()
    test_img, test_lab = next(test)
    labels.append(np.reshape(np.concatenate((np.array(test_lab)[0],
                                             np.array(test_lab)[1],
                                             np.array(test_lab)[2],
                                             np.array(test_lab)[3],
                                             np.array(test_lab)[4],
                                             np.array(test_lab)[5]),axis=0),(1,6))[-1,:])
    predic_tmp = model.predict(test_img)
    predic.append(np.reshape(np.concatenate((np.array(predic_tmp)[0],
                                             np.array(predic_tmp)[1],
                                             np.array(predic_tmp)[2],
                                             np.array(predic_tmp)[3],
                                             np.array(predic_tmp)[4],
                                             np.array(predic_tmp)[5]),axis=0),(1,6))[-1,:])
    print(time.time()-start)

1.621077060699463


In [11]:
test_img[:1,:2,...].shape

(1, 2, 384, 512, 6)